In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Enhanced Optimization for Radar and Communications Applications 


The Low Autocorrelation Binary Sequences (LABS) is an important and challenging optimization problem with applications related to radar, telecommunications, and other signal related applications. This CUDA-Q Academic module will focus on a clever quantum-enhanced hybrid method developed in a collaboration between Kipu Quantum, University of the Basque Country EHU, and NVIDIA for solving the LABS problem. (This notebook was jointly developed with input from all collaborators.)

Other CUDA-Q Academic modules like [Divide and Conquer MaxCut QAOA](https://github.com/NVIDIA/cuda-q-academic/tree/main/qaoa-for-max-cut) and [Quantum Finance](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb), demonstrate how quantum computing can be used outright to solve optimization problems. This notebook demonstrates a slightly different approach. Rather than considering QPUs as the tool to produce the final answer, it demonstrates how quantum can be used to enhance the effectiveness of leading classical methods.  

The benefits of such an approach were highlighted in [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1).  This notebook, co-created with the authors of the paper, will allow you to explore the findings of their research and write your own CUDA-Q code that builds a representative quantum-enhanced workflow for solving the LABS problem. Moreover, it will introduce advancements in counteradiabatic optimization techniques on which reduce the quantum resources required to run on a QPU.

**Prerequisites:** This lab assumes you have a basic knowledge of quantum computing, including operators, gates, etc.  For a refresher on some of these topics, explore the [Quick start to Quantum](https://github.com/NVIDIA/cuda-q-academic/tree/main/quick-start-to-quantum) series.

**In this lab you will:**
* 1. Understand the LABS problem and its relation ot radar and communication applications.
* 2. Solve LABS classically with memetic tabu search and learn about the limitations of such methods.
* 3. Code a couteradiabatic algorithm using CUDA-Q to produce approximate solutions to the LABS problem.
* 4. Use the CUDA-Q results to seed your tabu search and understand the potential benefits of this approach.


**Terminology you will use:**
* Low autocorrelation of binary sequences (LABS)
* counteradiabatic optimization
* memetic-tabu search

**CUDA-Q Syntax you will use:**
* cudaq.sample()
* @cudaq.kernel
* ry(), rx(), rz(), x(), h() 
* x.ctrl()

Run the code below to initialize the libraries you will need.

In [4]:
import cudaq
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils

## The LABS problem and applications

The **Low Autocorrelation Binary Sequences (LABS)** problem is fundamental to many applications, but originated with applications to radar. 

Consider a radar that monitors airport traffic.  The radar signal sent to detect incoming planes must have as much range as possible to ensure safe approaches are planned well in advance.  The range of a radar signal can be increased by sending a longer pulse.  However, in order to differentiate between multiple objects, pulses need to be short to provide high resolution. So, how do you handle situations where you need both?

One solution is a technique called pulse compression.  The idea is to send a long signal, but vary the phase at regular intervals such that the resolution is increased. Generally, the initial signal will encode a binary sequence of phase shifts, where each interval corresponds to a signal with a 0 or 180 degree phase shift. 

The tricky part is selecting an optimal encoding sequence.  When the signal returns, it is fed into a matched filter with the hope that a singular sharp peak will appear, indicating clear detection.  The autocorrelation of the original signal, or how similar the signal is to itself,  determines if a single peak or a messier signal with sidelobes will be detected. A signal should have high autocorrelation when overlayed on top of itself, but low autocorrelation when shifted with a lag. 

Consider the image below.  The signal on the left has a crisp single peak while the single on the right produces many undesirable sidelobes which can inhibit clear detection.  

<img src="images/quantum_enhanced_optimization_LABS/radar.png" width="800">


So, how do you select a good signal?   This is where LABS comes in, defining these questions as a binary optimization problem. Given a binary sequence of length $N$, $(s_1 \cdots s_N) \in {\pm 1}^N$, the goal is to minimize the following objective function.

$$ E(s) = \sum_{k=1}^{N-1} C_k^2 $$

Where $C_k$ is defined as. 

 $$C_k= \sum_{i=1}^{N-k} s_is_{i+k}$$


So, each $C_k$ computes how similar the original signal is to the shifted one for each offset value $k$.  To explore this more, try the interactive widget linked [here](https://nvidia.github.io/cuda-q-academic/interactive_widgets/labs_visualization.html).  See if you can select a very good and very poor sequence and see the difference for the case of $N=7$.

## Classical Solution of the LABS problem

The LABS problem is tricky to solve for a few reasons. First, the configuration space grows exponentially.  Second, underlying symmetries of the problem result in many degeneracies in the optimization landscape severely inhibiting local search methods. 

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 1:</h3>
    <p style="font-size: 16px; color: #333;">
Using the widget above, try to find some of the symmetries for the LABS problem. That is, for a fixed bitstring length, can you find patterns to produce the same energy with different pulse patterns. 
</div>

The best known performance for a classical optimization technique is Memetic Tabu search (MTS) which exhibits a scaling of $O(1.34^N)$.  The MTS algorithm is depicted below.  It begins with a randomly selected population of bitstrings and finds the best solution from them.  Then, a child is selected by sampling directly from or combining multiple bitstrings from the population.  The child is mutated with probability $p_{mutate}$ and then input to a tabu search, which performs a modified greedy local search starting from the child bitstring.  If the result is better than the best in the population, it is updated as the new leader and randomly replaces a  bitstring in the population.


<img src="images/quantum_enhanced_optimization_LABS/mts_algorithm.png" width="500">

Such an approach is fast, parallelizable, and allows for exploration with improved searching of the solution landscape.  

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 2:</h3>
    <p style="font-size: 16px; color: #333;">
Before exploring any quantum approach, get a sense for how MTS works by coding it yourself based generally on the figure above. Algorithms for the combine and mutate steps are provided below as used in the paper. You may need to research more specific details of the process, especially for how tabu search is performed. The MTS procedure should output and optimal bitstring and its energy.  Also, write a function to visualize the results including the energy distribution of the final population.
</div>



<img src="images/quantum_enhanced_optimization_LABS/combine_mutate.png" width="400">



In [ ]:
import numpy as np
import time
import unittest

# --- 1. Utility & Transformation Functions ---

def to_bits(s_pm1: np.ndarray) -> np.ndarray:
    return ((s_pm1 + 1) // 2).astype(np.int8)

def to_pm1(bits) -> np.ndarray:
    return (2 * np.array(bits, dtype=np.int8) - 1).astype(np.int8)

def string_to_array(s_str: str) -> np.ndarray:
    return np.array([int(b) for b in s_str], dtype=np.int8)

def bit_string(s_pm1: np.ndarray) -> str:
    return "".join(map(str, to_bits(s_pm1)))

def invert_bits(bits: np.ndarray) -> np.ndarray:
    """0 -> 1, 1 -> 0"""
    return 1 - bits

def symmetrize_bits(bits: np.ndarray) -> np.ndarray:
    """Reverse the sequence"""
    return bits[::-1]

# --- 2. Energy Logic ---

def get_correlations(s: np.ndarray) -> np.ndarray:
    n = s.size
    return np.array([np.dot(s[:-k], s[k:]) for k in range(1, n)], dtype=np.int32)

def calculate_energy(s: np.ndarray) -> int:
    corr = get_correlations(s).astype(np.int64)
    return int(np.sum(corr**2))

def get_delta_energy(s: np.ndarray, C: np.ndarray, E: int, j: int):
    n, sj = s.size, int(s[j])
    delta_c = np.zeros_like(C, dtype=np.int32)
    for k in range(1, n):
        d = 0
        if j + k < n: d += sj * s[j + k]
        if j - k >= 0: d += s[j - k] * sj
        if d != 0: delta_c[k-1] = -2 * d
    
    c64, dc64 = C.astype(np.int64), delta_c.astype(np.int64)
    new_e = E + int(np.sum(2 * c64 * dc64 + dc64 * dc64))
    return new_e, delta_c

# --- 3. Search Logic (Tabu & MTS) ---

def tabu_search(s0, max_iters, tenure, cand_size, rng, global_best_e):
    s = s0.copy()
    n = s.size
    curr_c, curr_e = get_correlations(s), calculate_energy(s)
    best_local_s, best_local_e = s.copy(), curr_e
    tabu_until = np.zeros(n, dtype=np.int32)

    for it in range(1, max_iters + 1):
        candidates = rng.choice(n, size=min(n, cand_size), replace=False)
        bj, be, bdc = None, None, None

        for j in candidates:
            en, dc = get_delta_energy(s, curr_c, curr_e, j)
            if tabu_until[j] > it and en >= global_best_e: continue
            if be is None or en < be: bj, be, bdc = j, en, dc
        
        if bj is None: # Fallback if all tabu
            for j in candidates:
                en, dc = get_delta_energy(s, curr_c, curr_e, j)
                if be is None or en < be: bj, be, bdc = j, en, dc

        s[bj] *= -1
        curr_c += bdc
        curr_e = be
        tabu_until[bj] = it + tenure + rng.integers(0, 5)
        if curr_e < best_local_e: best_local_e, best_local_s = curr_e, s.copy()

    return best_local_s, best_local_e

def run_mts_labs(n, target_energy=0, pop_size=32, mts_iters=500, tabu_iters=300, seed=42, initial_pop=None):
    rng = np.random.default_rng(seed)
    start_t = time.time()
    if initial_pop is None:
        pop = rng.choice(np.array([-1, 1], dtype=np.int8), size=(pop_size, n))
    else:
        assert initial_pop.shape == (pop_size, n)
        pop = np.array(initial_pop)
    pop_e = np.array([calculate_energy(ind) for ind in pop], dtype=np.int64)
    best_idx = np.argmin(pop_e)
    best_s, best_e = pop[best_idx].copy(), int(pop_e[best_idx])

    for it in range(mts_iters):
        if best_e <= target_energy: break
        
        # Crossover or Mutation
        if rng.random() < 0.7:
            i1, i2 = rng.integers(0, pop_size, size=2)
            child = np.concatenate([pop[i1][:n//2], pop[i2][n//2:]])
        else:
            child = pop[rng.integers(0, pop_size)].copy()
            mask = rng.random(n) < 0.02
            child[mask] *= -1

        res_s, res_e = tabu_search(child, tabu_iters, 30, 64, rng, best_e)
        
        ridx = rng.integers(0, pop_size)
        if res_e < pop_e[ridx]: pop[ridx], pop_e[ridx] = res_s, res_e

        if res_e < best_e:
            best_e, best_s = res_e, res_s.copy()
            print(f"{time.time()-start_t:<8.2f}s | E: {best_e:<6} | {bit_string(best_s)}")

    return best_s, best_e

# --- 4. Unified Test Suite ---

class TestLABS(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.KNOWN_ENERGIES = {
            3: 1, 4: 2, 5: 2, 6: 7, 7: 3, 8: 8, 9: 12, 10: 13, 11: 5,
            12: 10, 13: 6, 14: 19, 15: 15, 16: 24, 17: 32, 18: 25, 19: 29,
            20: 26, 21: 26, 22: 39, 23: 47, 24: 36, 25: 36, 26: 45, 27: 37,
            28: 50, 29: 62, 30: 59, 31: 67, 32: 64, 33: 64, 34: 65, 35: 73,
            36: 82, 37: 86, 38: 87, 39: 99, 40: 108, 41: 180, 42: 101,
            43: 109, 44: 122, 45: 118, 46: 131, 47: 135, 48: 140, 49: 136,
            50: 153, 51: 153, 52: 166, 53: 170, 54: 175, 55: 171, 56: 192,
            57: 188, 58: 197, 59: 205, 60: 218, 61: 226, 62: 235, 63: 207,
            64: 208, 65: 240, 66: 257
        }

    def test_specific_sequences_and_variants(self):
        """Tests the 3 requested strings and their inverted/symmetric versions."""
        cases = [
            ("0100111", 3),
            ("1111000", 35),
            ("1011100", 7)
        ]
        for s_str, expected_e in cases:
            bits = string_to_array(s_str)
            variants = {
                "Original": bits,
                "Inverted": invert_bits(bits),
                "Symmetric": symmetrize_bits(bits),
                "Inv-Sym": invert_bits(symmetrize_bits(bits))
            }
            for name, v_bits in variants.items():
                with self.subTest(string=s_str, variant=name):
                    e = calculate_energy(to_pm1(v_bits))
                    self.assertEqual(e, expected_e)

    def test_invariance_properties(self):
        """Property tests using random sequences."""
        rng = np.random.default_rng(42)
        s = rng.choice([-1, 1], size=20).astype(np.int8)
        e_orig = calculate_energy(s)
        self.assertEqual(e_orig, calculate_energy(s * -1))
        self.assertEqual(e_orig, calculate_energy(s[::-1]))

    def test_known_minima_small_n(self):
        """Integration test: Verify solver finds known minima for N=3 to N=20."""
        print("\nRunning Integration Tests (N=3 to 20)...")
        for n in range(3, 21):
            print(f"Starting N = {n}")
            target = self.KNOWN_ENERGIES[n]
            with self.subTest(n=n):
                _, best_e = run_mts_labs(n=n, target_energy=target, mts_iters=100)
                self.assertEqual(best_e, target)

# --- Execution ---
if __name__ == '__main__':
    # Run unittests in Jupyter
    unittest.main(argv=['first-arg-is-ignored'], exit=False, buffer=True)

## Building a Quantum Enhanced Workflow

Despite the effectiveness of MTS, it still exhibits exponential scaling  $O(1.34^N)$ behavior and becomes intractable for large $N$.  Quantum computing provides a potential alternative method for solving the LABS problem because the properties of entanglement, interference, and superpositon may allow for a better global search.  Recent demonstrations have even produced evidence that the quantum approximate optimization algorithm (QAOA) can be used to reduce the scaling of the LABS problem to $O(1.21^N)$ for $N$ between 28 and 40 with quantum minimum finding.

However, current quantum hardware limitations restrict solution to problems of greater than about $N=20$, meaning that it will be some time before quantum approaches can outperform the classical state of the art. It should also be noted that standard QAOA can struggle with LABS and require many layers to converge the parameters if other tricks are not employed.

The authors of [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1) cleverly explored an alternate path that combines quantum and classical approaches and might be able to provide a more near-term benefit.  Instead of expecting the quantum computer to solve the problem entirely, they asked how a quantum approach might enhance MTS.

The basic idea is that a quantum optimization routine could run first and the resulting state be sampled to produce a better population for MTS. Many such heuristics for defining the initial population are possible, but the rest of this notebook will explore their methodology, help you to build the workflow yourself, and allow you to analyze the benefits of their approach.

The first step of quantum enhanced MTS (QE-MTS) is to prepare a circuit with CUDA-Q that approximates the ground state of the Hamiltonian corresponding to the LABS problem. You could do this with any optimization algorithm such as QAOA or using an adiabatic approach.  (See the [Quantum Portfolio Optimization](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb) CUDA-Q Academic lab for a detailed comparison of these two common approaches.)

The authors of this work opted for an adiabatic approach (More on why later). Recall that the goal of an adiabatic optimization is to begin with a Hamiltonian that has an easily prepared ground state ($H_i$). Then, the adiabatic Hamiltonian $H_{ad}$ can be constructed as $H_{ad}(\lambda) = (1-\lambda)H_i +\lambda H_f $, where $\lambda$ is a function of time and $H_f$ is the Hamiltonian representing a qubit encoding of the LABS problem. 

$$H_f = 2 \sum_{i=1}^{N-2} \sigma_i^z \sum_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} \sigma_{i+k}^z 
+ 4 \sum_{i=1}^{N-3} \sigma_i^z \sum_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \sum_{k=t+1}^{N-i-t} \sigma_{i+t}^z \sigma_{i+k}^z \sigma_{i+k+t}^z$$

The authors also selected $H_i = \sum_i h^x_i \sigma^x_i $ which has an easily prepared ground state of $\ket{+}^{\otimes N}$.

The challenge for implementing the optimization procedure becomes selection of an operator that will quickly and accurately evolve to the ground state of $H_f$.  One approach is to use a so-called auxiliary countradiabatic (CD) term $H_{CD}$, which corrects diabatic transitions that jump out of the ground state during the evolution. The figure below demonstrates the benefit of using a CD correction.


<img src="images/quantum_enhanced_optimization_LABS/counteradiabatic.png" width="900">




An operator called the adiabatic gauge potential $A_{\lambda}$ is the ideal choice for the CD term as it suppresses all possible diabatic transitions, resulting in the following total system to evolve.

$$ H(\lambda) = H_{ad}(\lambda) + \lambda H_{CD} (\lambda) $$

$A(\lambda)$ is derrived from $H_{ad}(\lambda)$  (see paper for details) as it contains information about underlying physics of the problem. 

There is a problem though.  The $A(\lambda)$ term cannot be efficiently expressed exactly and needs to be approximated.  It also turns out that in the so called impulse regime, where the adiabatic evolution is very fast, $H_{cd} (\lambda)$ dominates $H_{ad}(\lambda)$, meaning that the final implementation corresponds to the operator $H(\lambda) = H^1_{cd}(\lambda)$ where  $H^1_{cd}(\lambda)$ is a first order approximation of $A(\lambda)$ (see equation 7 in the paper).

A final step is to use Trotterization to define the quantum circuit to apply $e^{-\theta (t) i H_{cd}}$. The details for this derivation are shown in the appendix of the paper. and result from equation B3 is shown below.  

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

It turns out that this implementation is more efficient than QAOA in terms of gate count. The authors calculated that for $N=67$, QAOA would require 1.4 million entangling gates while the CD approach derived here requires only 236 thousand entangling gates.


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 3:</h3>
    <p style="font-size: 16px; color: #333;">
At first glance, this equation might looks quite complicated. However, observe the structure and note two "blocks" of terms.  Can you spot them?  

They are 2 qubit terms that look like $R_{YZ}(\theta)$ or $R_{ZY}(\theta)$.

As well as 4 qubit terms that look like $R_{YZZZ}(\theta)$, $R_{ZYZZ}(\theta)$, $R_{ZZYZ}(\theta)$, or $R_{ZZZY}(\theta)$.

Thankfully the authors derive a pair of circuit implementations for the two and four qubit terms respectively, shown in the figures below.

Using CUDA-Q, write a kernel for each which will be used later to construct the full implementation.

* Hint: Remember that the adjoint of a rotation gate is the same as rotating in the opposite direction. 

* Hint: You may also want to define a CUDA-Q kernel for an R$_{ZZ}$ gate.

* Hint: Implementing a circuit from a paper is a great place where AI can help accelerate your work.  If you have access to a coding assistant, feel free to use it here.
</div>

<img src="images/quantum_enhanced_optimization_LABS/kernels.png" width="1300">


In [ ]:
@cudaq.kernel
def qc(n: int, indices: list[int], indices2: list[int], theta_list: list[float]):
    qubits = cudaq.qvector(n)
    
    # Apply gates
    for i in range(n):
        h(qubits[i])

    for t in range(len(theta_list)):
        for i in range(len(indices) // 2):
            i1 = indices[2*i]
            i0 = indices[2*i + 1]
    
            rx(np.pi/2, qubits[i0])
    
            cx(qubits[i0], qubits[i1])
            rz(theta_list[t], qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            rx(-np.pi/2, qubits[i0])
            rx(np.pi/2, qubits[i1])
    
            cx(qubits[i0], qubits[i1])
            rz(theta_list[t], qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            rx(-np.pi/2, qubits[i1])
    
        for i in range(len(indices2) // 4):
            i0 = indices2[4*i]
            i1 = indices2[4*i + 1]
            i2 = indices2[4*i + 2]
            i3 = indices2[4*i + 3]
    
            rx(-np.pi/2, qubits[i0])
            ry(np.pi/2, qubits[i1])
            ry(-np.pi/2, qubits[i2])
    
            cx(qubits[i0], qubits[i1])
            rz(-np.pi/2, qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            cx(qubits[i2], qubits[i3])
            rz(-np.pi/2, qubits[i3])
            cx(qubits[i2], qubits[i3])
    
            rx(np.pi/2, qubits[i0])
            ry(-np.pi/2, qubits[i1])
            ry(np.pi/2, qubits[i2])
            rx(-np.pi/2, qubits[i3])
    
            rx(-np.pi/2, qubits[i1])
            rx(-np.pi/2, qubits[i2])
    
            cx(qubits[i1], qubits[i2])
            rz(theta_list[t], qubits[i2])
            cx(qubits[i1], qubits[i2])
    
            rx(np.pi/2, qubits[i1])
            rx(np.pi, qubits[i2])
    
            ry(np.pi/2, qubits[i1])
    
            cx(qubits[i0], qubits[i1])
            rz(np.pi/2, qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            rx(np.pi/2, qubits[i0])
            ry(-np.pi/2, qubits[i1])
    
            cx(qubits[i1], qubits[i2])
            rz(-theta_list[t], qubits[i2])
            cx(qubits[i1], qubits[i2])
    
            rx(np.pi/2, qubits[i1])
            rx(-np.pi, qubits[i2])
    
            cx(qubits[i1], qubits[i2])
            rz(-theta_list[t], qubits[i2])
            cx(qubits[i1], qubits[i2])
            
            rx(-np.pi, qubits[i1])
            ry(np.pi/2, qubits[i2])
    
            cx(qubits[i2], qubits[i3])
            rz(-np.pi/2, qubits[i3])
            cx(qubits[i2], qubits[i3])
    
            ry(-np.pi/2, qubits[i2])
            rx(-np.pi/2, qubits[i3])
    
            rx(-np.pi/2, qubits[i2])
    
            cx(qubits[i1], qubits[i2])
            rz(theta_list[t], qubits[i2])
            cx(qubits[i1], qubits[i2])
    
            rx(np.pi/2, qubits[i1])
            rx(np.pi/2, qubits[i2])
            
            ry(-np.pi/2, qubits[i1])
            ry(np.pi/2, qubits[i2])
    
            cx(qubits[i0], qubits[i1])
            rz(np.pi/2, qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            cx(qubits[i2], qubits[i3])
            rz(np.pi/2, qubits[i3])
            cx(qubits[i2], qubits[i3])
    
            ry(np.pi/2, qubits[i1])
            ry(-np.pi/2, qubits[i2])
            rx(np.pi/2, qubits[i3])

    mz(qubits)

There are a few additional items we need to consider before completing the final implementation of the entire circuit.  One simplification we can make is that for our problem the $h_i^x$ terms are all 1 and any $h_b^x$ terms are 0, and are only there for generalizations of this model. 

The remaining challenge is derivation of the angles that are used to apply each of the circuits you defined above. These are obtained from two terms $\lambda(t)$ and $\alpha(t)$.  


The $\lambda(t)$ defines an annealing schedule and is generally a Sin function which slowly "turns on" the problem Hamiltonian.  For computing our angles, we need the derivative of $\lambda(t)$.

The $\alpha$ term is a bit trickier and is the solution to a set of differential equations which minimize the distance between $H^1_{CD}(\lambda)$ and $A(\lambda)$.  The result is 

$$\alpha(t) = \frac{-\Gamma_1(t)}{\Gamma_2(t)} $$

Where $\Gamma_1(t)$ and $\Gamma_2(t)$ are defined in equations 16 and 17 of the paper and essentially depend on the structure of the optimization problem.  Curious learners can look at the functions in `labs_utils.py`  to see how these are computed, based on the problem size and specific time step in the Trotter process. 


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 4:</h3>
    <p style="font-size: 16px; color: #333;">
The `compute_theta` function, called in the following cells, requires all indices of the two and four body terms. These will be used again in our main kernel to apply the respective gates.  Use the products in the formula below to finish the function in the cell below.  Save them as `G2` and `G4` where each is a list of lists of indices defining the two and four term interactions. As you are translating an equation to a set of loops, this is a great opportunity to use an AI coding assistant.

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

</div>


In [ ]:
def get_interactions(n):
    """
    Generates the interaction sets G2 and G4 based on the loop limits in Eq. 15.
    Returns standard 0-based indices as lists of lists of ints.
    
    Args:
        N (int): Sequence length.
        
    Returns:
        G2: List of lists containing two body term indices
        G4: List of lists containing four body term indices
    """
    
    pairs = []
    for i in range(1,n):
        square = []
        for j in range(n-i):
            square += [(i+j,j)]
        
        pairs += [square]

    full = []
    for x in range(len(pairs)):
        test = pairs[x]
        for i in range(len(test)):
            for j in range(len(test)):
                list1 = [test[i][0], test[i][1], test[j][0], test[j][1]]
                set1 = set()
                for w in list1:
                    if list1.count(w) == 1:
                        set1.add(w)
                if set1 != set():
                    full += [set1]

    unique = [set(s) for s in set(frozenset(s) for s in full)]

    list_pairs = []
    for i in unique:
        list_pairs += [list(i)]

    list_2 = []
    list_4 = []
    G2 = []
    G4 = []

    for i in list_pairs:
        if len(i) == 2:
            list_2 += [i[0], i[1]]
            G2 += [i]
        else:
            list_4 += [i[0], i[1], i[2], i[3]]
            G4 += [i]

                
    return G2, G4, list_2, list_4



<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 5:</h3>
    <p style="font-size: 16px; color: #333;">
You are now ready to construct the entire circuit and run the counteradiabatic optimization procedure. The final kernel needs to apply Equation 15 for a specified total evolution time $T$ and the `n_steps` number of Trotter steps.  It must also take as input, the indices for the two and four body terms and the thetas to be applied each step, as these cannot be computed within a CUDA-Q kernel.

The helper function `compute_theta` computes the theta parameters for you, using a few additional functions in accordance with the equations defined in the paper.
</div>


In [ ]:


T=1               # total time
n_steps = 1       # number of trotter steps
dt = T / n_steps
N = 20
G2, G4, list_2, list_4 = get_interactions(N)

thetas =[]

for step in range(1, n_steps + 1):
    t = step * dt
    theta_val = utils.compute_theta(t, dt, T, N, G2, G4)
    thetas.append(theta_val)

## Generating Quantum Enhanced Results

Recall that the point of this lab is to demonstrate the potential benefits of running a quantum subroutine as a preprocessing step for classical optimization of a challenging problem like LABS. you now have all of the tools you need to try this for yourself.

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 6:</h3>
    <p style="font-size: 16px; color: #333;">
Use your CUDA-Q code to prepare an initial population for your memetic search algorithm and see if you can improve the results relative to a random initial population.  If you are running on a CPU, you will need to run smaller problem instances. The code below sets up the problem

</div>


In [ ]:
import numpy as np

def compute_topology_overlaps(G2, G4):
    """
    Computes the topological invariants I_22, I_24, I_44 based on set overlaps.
    I_alpha_beta counts how many sets share IDENTICAL elements.
    """
    # Helper to count identical sets
    def count_matches(list_a, list_b):
        matches = 0
        # Convert to sorted tuples to ensure order doesn't affect equality
        set_b = set(tuple(sorted(x)) for x in list_b)
        for item in list_a:
            if tuple(sorted(item)) in set_b:
                matches += 1
        return matches

    # For standard LABS/Ising chains, these overlaps are often 0 or specific integers
    # We implement the general counting logic here.
    I_22 = count_matches(G2, G2) # Self overlap is just len(G2)
    I_44 = count_matches(G4, G4) # Self overlap is just len(G4)
    I_24 = 0 # 2-body set vs 4-body set overlap usually 0 as sizes differ
    
    return {'22': I_22, '44': I_44, '24': I_24}

from math import sin, cos, pi

def compute_theta(t, dt, total_time, N, G2, G4):
    """
    Computes theta(t) using the analytical solutions for Gamma1 and Gamma2.
    """
    
    # ---  Better Schedule (Trigonometric) ---
    # lambda(t) = sin^2(pi * t / 2T)
    # lambda_dot(t) = (pi / 2T) * sin(pi * t / T)
    
    if total_time == 0:
        return 0.0

    # Argument for the trig functions
    arg = (pi * t) / (2.0 * total_time)
    
    lam = sin(arg)**2
    # Derivative: (pi/2T) * sin(2 * arg) -> sin(pi * t / T)
    lam_dot = (pi / (2.0 * total_time)) * sin((pi * t) / total_time)
    
    
    # ---  Calculate Gamma Terms (LABS assumptions: h^x=1, h^b=0) ---
    # For G2 (size 2): S_x = 2
    # For G4 (size 4): S_x = 4
    
    # Gamma 1 (Eq 16)
    # Gamma1 = 16 * Sum_G2(S_x) + 64 * Sum_G4(S_x)
    term_g1_2 = 16 * len(G2) * 2
    term_g1_4 = 64 * len(G4) * 4
    Gamma1 = term_g1_2 + term_g1_4
    
    # Gamma 2 (Eq 17)
    # G2 term: Sum (lambda^2 * S_x)
    # S_x = 2
    sum_G2 = len(G2) * (lam**2 * 2)
    
    # G4 term: 4 * Sum (4*lambda^2 * S_x + (1-lambda)^2 * 8)
    # S_x = 4
    # Inner = 16*lam^2 + 8*(1-lam)^2
    sum_G4 = 4 * len(G4) * (16 * (lam**2) + 8 * ((1 - lam)**2))
    
    # Topology part
    I_vals = compute_topology_overlaps(G2, G4)
    term_topology = 4 * (lam**2) * (4 * I_vals['24'] + I_vals['22']) + 64 * (lam**2) * I_vals['44']
    
    # Combine Gamma 2
    Gamma2 = -256 * (term_topology + sum_G2 + sum_G4)

    # ---  Alpha & Theta ---
    if abs(Gamma2) < 1e-12:
        alpha = 0.0
    else:
        alpha = - Gamma1 / Gamma2
        
    return dt * alpha * lam_dot


import cudaq
import time

n = 10

pairs = []
for i in range(1,n):
    square = []
    for j in range(n-i):
        square += [(i+j,j)]
    
    pairs += [square]

full = []
for x in range(len(pairs)):
    test = pairs[x]
    for i in range(len(test)):
        for j in range(len(test)):
            list1 = [test[i][0], test[i][1], test[j][0], test[j][1]]
            set1 = set()
            for w in list1:
                if list1.count(w) == 1:
                    set1.add(w)
            if set1 != set():
                full += [set1]

unique = [set(s) for s in set(frozenset(s) for s in full)]

list_pairs = []
for i in unique:
    list_pairs += [list(i)]

list_2 = []
list_4 = []
G2 = []
G4 = []

for i in list_pairs:
    if len(i) == 2:
        list_2 += [i[0], i[1]]
        G2 += [i]
    else:
        list_4 += [i[0], i[1], i[2], i[3]]
        G4 += [i]
    

T=1               # total time
n_steps = 1       # number of trotter steps
dt = T / n_steps
N = n

thetas =[]

for step in range(1, n_steps + 1):
    t = step * dt
    theta_val = compute_theta(t, dt, T, N, G2, G4)
    thetas.append(theta_val)



string_list = []
for i in range(100):
    result = cudaq.sample(qc, n, list_2, list_4, thetas, shots_count=100000)

    most_probable = max(result.items(), key=lambda x: x[1])
    bitstring, count = most_probable
    string_list += [bitstring]

In [ ]:
# MTS with quantum-sampled initial population
initial_pop = []
for s in string_list:
    bits = string_to_array(s)
    pm1 = to_pm1(bits)
    initial_pop += [pm1]
# initial_pop = np.array(initial_pop)
best_s, best_e = run_mts_labs(n=n, target_energy=0, pop_size=100, mts_iters=500, tabu_iters=300, seed=42, initial_pop=initial_pop)

The results clearly show that a population sampled from CUDA-Q results in an improved distribution and a lower energy final result. This is exactly the goal of quantum enhanced optimization.  To not necessarily solve the problem, but improve the effectiveness of state-of-the-art classical approaches. 

A few major caveats need to be mentioned here. First, We are comparing a quantum generated population to a random sample.  It is quite likely that other classical or quantum heuristics could be used to produce an initial population that might even beat the counteradiabatic method you used, so we cannot make any claims that this is the best. 

Recall that the point of the counteradiabatic approach derived in the paper is that it is more efficient in terms of two-qubit gates relative to QAOA. The benefits of this regime would only truly come into play in a setting (e.g. larger problem instance) where it is too difficult to produce a good initial population with any know classical heuristic, and the counteradiabatic approach is more efficiently run on a QPU compared to alternatives.

We should also note that we are comparing a single sample of each approach.  Maybe the quantum sample got lucky or the randomly generated population was unlucky and a more rigorous comparison would need to repeat the analysis many times to draw any confidently conclusions.  

The authors of the paper discuss all of these considerations, but propose an analysis that is quite interesting related to the scaling of the technique. Rather than run large simulations ourselves, examine their results below. 


<img src="images/quantum_enhanced_optimization_LABS/tabu_search_results.png" width="900">

The authors computed replicate median (median of solving the problem repeated with same setup) time to solutions (excluding time to sample from QPU) for problem sizes $N=27$ to $N=37$. Two interesting conclusions can be drawn from this. First, standard memetic tabu search (MTS) is generally faster than quantum enhanced (QE) MTS.  But there are two promising trends. For larger problems, the QE-MTS experiments occasionally have excellent performance with times to solution much smaller than all of the MTS data points.  These outliers indicate there are certain instances where QE-MTS could provide much faster time-to-solution. 

More importantly, if a line of best fit is calculated using the median of each set of medians, the slope of the QE-MTS line is smaller than the MTS!  This seems to indicate that QE solution of this problem scales $O(1.24^N)$ which is better than the best known classical heuristic ($O(1.34^N)$) and the best known quantum approach (QAOA - $O(1.46^N)$).

For problems of size of $N=47$ or greater, the authors anticipate that QE-MTS could be a promising technique and produce good initial populations that are difficult to obtain classically. 

The study reinforces the potential of hybrid workflows enhanced by quantum data such that a classical routine is still the primary solver, but quantum computers make it much more effective.  Future work can explore improvements to both the quantum and classical sides, such as including GPU accelerated memetic search on the classical side.

## Self-validation To Be Completed for Phase 1

In this section, explain how you verified your results. Did you calculate solutions by hand for small N? Did you create unit tests? Did you cross-reference your Quantum energy values against your Classical MTS results? Did you check known symmetries?